In [1]:
%matplotlib inline

import sys
sys.path.append("..") # This allows for importing from other directories above
# Our imports
from models.adaboost_mh import AdaBoostMH
from models.weak_learner import stump_base as weak_clf
# Standard Imports
import numpy as np
import matplotlib.pyplot as plt


# Load the data
switch = 2
if switch == 0:
    X_train = np.load('../data/pendigits/pendigits_train_data.npy')
    y_train = np.load('../data/pendigits/pendigits_train_labels.npy')
    X_test = np.load('../data/pendigits/pendigits_test_data.npy')
    y_test = np.load('../data/pendigits/pendigits_test_labels.npy')
    k = 10
elif switch == 1:
    X_train = np.load('../data/synth/synth_train_data.npy')
    y_train = np.load('../data/synth/synth_train_labels.npy')
    X_test = np.load('../data/synth/synth_test_data.npy')
    y_test = np.load('../data/synth/synth_test_labels.npy')
    k = 10
elif switch == 2:
    X_train = np.load('../data/synthBinary/synthBinary_train_data.npy')
    y_train = np.load('../data/synthBinary/synthBinary_train_labels.npy')
    X_test = np.load('../data/synthBinary/synthBinary_test_data.npy')
    y_test = np.load('../data/synthBinary/synthBinary_test_labels.npy')
    k = 2
elif switch == 3:
    X_train = np.load('../data/synthTernary/synthTernary_train_data.npy')
    y_train = np.load('../data/synthTernary/synthTernary_train_labels.npy')
    X_test = np.load('../data/synthTernary/synthTernary_test_data.npy')
    y_test = np.load('../data/synthTernary/synthTernary_test_labels.npy')
    k = 3
else:
    X_train = np.load('../data/synthQuaternary/train_data.npy')
    y_train = np.load('../data/synthQuaternary/train_labels.npy')
    X_test = np.load('../data/synthQuaternary/test_data.npy')
    y_test = np.load('../data/synthQuaternary/test_labels.npy')
    k = 4
    
#y_train = y_train[0:6]
#X_train = X_train[0:6]
    
# Initialize model 
model = AdaBoostMH(X_train, y_train, X_test, y_test, 0.8)

In [2]:
y_train

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [3]:
X_train

array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40]])

In [4]:
T = 5
verbose = 2

In [5]:
err_tr, err_te, gams, v_ts, d_ts = model.run_factorized(T, weak_clf, 'bal', verbose)

Round 1
(40, 2)
alpha is 0.050041729278491216
Edge is 0.05
Energy is 0.998749217771909
v is [-1.  1.]
b is 1.5
 col is 0
Round 2
(40, 2)
alpha is 0.09552761838135461
Edge is 0.09523809523809515
Energy is 0.9954545219222319
v is [-1.  1.]
b is 39.5
 col is 0
Round 3
(40, 2)
alpha is 0.08717669357238877
Edge is 0.08695652173913052
Energy is 0.9962121075990956
v is [-1.  1.]
b is 1.5
 col is 0
Round 4
(40, 2)
alpha is 0.08017132503758954
Edge is 0.07999999999999985
Energy is 0.996794863550169
v is [-1.  1.]
b is 39.5
 col is 0
Round 5
(40, 2)
alpha is 0.0742100025591367
Edge is 0.0740740740740741
Energy is 0.9972527420619451
v is [-1.  1.]
b is 1.5
 col is 0


In [6]:
sch_err_tr, sch_err_te, gams, d_ts = model.run_kegl(T, weak_clf, 'bal', verbose)

Round 1
(40, 1)
(40, 1)


NameError: name 'i' is not defined

In [7]:
X = np.array([[x for x in range(5)] for y in range(3)])

In [8]:
X

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [18]:
np.atleast_2d(X[:,3]).T

array([[4],
       [4],
       [4],
       [4],
       [4]])

In [ ]:
print(err_tr)

In [ ]:
print(err_te)

In [ ]:
len(d_ts)

## Plot Pseudo Weights

In [ ]:
# One hot encode 
def one_hot_labels(y,k):
    """
    y: N by 1 numpy-array corresponding to the labels.
    k: float, corresponding to number of unique classes.
    returns: Y, an N by k numpy-array with each row containing
             a 1 for the correct class and -1 otherwise.
    """
    N = y.shape[0]
    Y = np.ones((N, k)) * -1
    for i in range(N):
        Y[i, y[i]] *= -1 # Make the correct class become +1.
    return Y

In [ ]:
Y_one_hot = one_hot_labels(y_train, k)

In [ ]:
def calc_pseudo_ws(Wts, vts, Y, T):
    w_pseudo_ts = []
    for t in range(T):
        #W_t = Wts[t + 1] # skip W_0
        W_t = Wts[t]
        v_t = vts[t] 
        # Get all the y_i,l * v_l in one matrix
        Y_mult_v_t = np.multiply(Y, v_t)
        # The above is either 1 or -1, to get an indicator
        # about whether their product is +1 or -1 you can 
        # add one to the whole matrix and all the -1 -> 0,
        # or add negative one to the whole matrix and all
        # the +1 -> 0. You just need to normalize by \pm 1/2,
        # to get an indicator or a matrix mask.
        w_p_mask = ((Y_mult_v_t + 1) * 0.5).astype('int')
        w_n_mask = ((Y_mult_v_t - 1) * -0.5).astype('int')
        w_p_t = np.sum(np.multiply(W_t, w_p_mask), axis=1)
        w_n_t = np.sum(np.multiply(W_t, w_n_mask), axis=1)
        w_pseudo_t = np.sum(np.abs(w_p_t - w_n_t))
        w_pseudo_ts.append(w_pseudo_t)
    return w_pseudo_ts

In [ ]:
pseudo_W_ts = calc_pseudo_ws(d_ts, v_ts, Y_one_hot, T)

In [ ]:
# Function for plotting Pseudo weights on same plot for two datasets
def make_plot(data_to_plot, T, ylabel, xlabel, title):
    ts = np.linspace(1, T, num=T)
    plt.plot(ts, data_to_plot) #, label='Pseudo Weights')
    #plt.legend(loc='best')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

In [ ]:
make_plot(pseudo_W_ts, T, "Pseudo Weight Value", "T", "Results")

In [ ]:
np.min(pseudo_W_ts)

In [ ]:
pseudo_W_ts[0]

## Plot training and test error

In [ ]:
make_plot(err_tr, T, "Training Error", "T", "Results")